In [1]:
import pandas as pd
import glob
import queue
import threading
import tkinter.messagebox
from tkinter import filedialog
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import *
from tkinter import font as tkFont
import os
import numpy as np
import random
import math
from datetime import datetime, timedelta, date

    
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Inventory Optimization")
        self.geometry('700x400')
        self.configure(bg = 'white smoke')
        self.main = tk.Frame(self) 
        self.main.configure(bg = 'white smoke')
        self.main.pack()
        helv10 = tkFont.Font(family='Helvetica', size=10, weight=tkFont.BOLD)
        
        #Input Data
        self.Data = tk.Label(self.main, text = "Input Data Folder", bg = 'white smoke')
        self.Data.grid(row=2, column = 0)
        entryText = tk.StringVar()
        self.DataFolder= tk.Entry(self.main, textvariable=entryText, width = 50)
        self.DataFolder.grid(row=2, column = 1, padx=30, pady=30)
        self.Select = tk.Button(self.main,
                                text = 'Select',
                                command=lambda:[self.get_path(entryText)], width = 20, bg = 'gainsboro')
        self.Select.grid(row=2, column = 2)
        
        #Very Low Consumption Analysis Paramters
        #Titles
        self.Title = tk.Label(self.main, text = "Very Low Consumption Analysis Parameters", 
                              bg = 'white smoke', font = helv10).grid(row=5, column = 1)  
        helv8 = tkFont.Font(family='Helvetica', size=8, slant = "italic")
        self.Note = tk.Label(self.main, text = "Enter weights in percentages adding to 100%", 
                             bg = 'white smoke', font = helv8).grid(row=6, column = 1)
       
        #Input paramters
        self.Func = tk.Label(self.main, text = "Functionality", bg = 'white smoke').grid(row=7, column = 0,  padx=3, pady=3)
        FuncText = tk.StringVar()
        self.entrFunc= tk.Entry(self.main, textvariable=FuncText, width = 40)
        self.entrFunc.grid(row=7, column = 1)
        self.LeadTime = tk.Label(self.main, text = "Lead Time", bg = 'white smoke').grid(row=8, column = 0,  padx=3, pady=3)
        LeadText = tk.StringVar()
        self.entrLead = tk.Entry(self.main,textvariable=LeadText , width = 40 )
        self.entrLead.grid(row=8, column = 1)
        self.Commodity = tk.Label(self.main, text = "Commodity/OEM", bg = 'white smoke').grid(row=9, column = 0, padx=3, pady=3)
        ComText = tk.StringVar()
        self.entrCom = tk.Entry(self.main, textvariable=ComText, width = 40)
        self.entrCom.grid(row=9, column = 1)
        self.LifeCycle = tk.Label(self.main, text = "Life Cycle", bg = 'white smoke').grid(row=10, column = 0, padx=3, pady=3)
        LifeText = tk.StringVar()
        self.entrLife = tk.Entry(self.main, textvariable=LifeText , width = 40)
        self.entrLife.grid(row=10, column = 1)
        self.Demand = tk.Label(self.main, text = "Demand (XYZ)", bg = 'white smoke').grid(row=11, column = 0,  padx=3, pady=3)
        DemandText = tk.StringVar()
        self.entrDemand = tk.Entry(self.main,  textvariable=DemandText, width = 40)
        self.entrDemand.grid(row=11, column = 1)
        self.Price = tk.Label(self.main, text = "Price (ABC)",bg = 'white smoke').grid(row=12, column = 0, padx=3, pady=3)
        PriceText = tk.StringVar()
        self.entrPrice = tk.Entry(self.main,  textvariable=PriceText, width = 40)
        self.entrPrice.grid(row=12, column = 1)
        self.Response = tk.Label(self.main, text = "Response Time", bg = 'white smoke').grid(row=13, column = 0, padx=3, pady=3)
        RespText = tk.StringVar()
        self.entrResp = tk.Entry(self.main,textvariable=RespText, width = 40)
        self.entrResp.grid(row=13, column = 1)
       
        #Default Button
        self.button1 = tk.Button(self.main,text = 'Use Default Weights', 
                                 command = lambda:self.set_default(FuncText, LeadText,ComText, LifeText, DemandText, PriceText, RespText), 
                                 bg = 'gainsboro', width = 20, fg = "firebrick2").grid(row=6, column = 2,  padx=5, pady=5)
        
        #Submit Button
        self.Submit = tk.Button(self.main,text = 'Submit', 
                                command = self.analysis_page, width = 20, 
                                bg = 'gainsboro').grid(row=15, column = 1)
    
    def get_path(self, entryText):
        folderpath = filedialog.askdirectory()
        entryText.set( folderpath )
    
    def analysis_page(self):
        #Save input
        self.data_path = self.DataFolder.get()
        self.Func = self.entrFunc.get()
        self.Lead = self.entrLead.get()
        self.Commodity = self.entrCom.get()
        self.Life = self.entrLife.get()
        self.Demand = self.entrDemand.get() 
        self.Price = self.entrPrice.get()
        self.Resp = self.entrResp.get()
        
        inventory_files = self.data_path + "\Inventory*"
        usage_files = self.data_path + r"\Issues*"

        if self.data_path == "":
            tkinter.messagebox.showinfo("Warning","Please select an input folder")
        elif self.Func == "" or self.Lead=="" or self.Commodity== "" or self.Life=="" or self.Life=="" or self.Demand=="" or self.Price=="" or self.Resp=="":
            tkinter.messagebox.showinfo("Warning","Please select weights")
        elif not glob.glob(inventory_files) or not glob.glob(usage_files):
            tkinter.messagebox.showinfo("Warning","No data files found in selected folder")    
        else:    
            #Create new window
            global main, root
            self.main.destroy()
            self.main=Frame(self)
            self.main.configure(bg = 'white smoke')
            self.main.pack()
            helv10 = tkFont.Font(family='Helvetica', size=10, weight=tkFont.BOLD)
            self.queue = queue.Queue()
            self.button = tk.Button(self.main,text = 'Run Analysis', 
                                     command = self.start_action, 
                                     width = 20, bg = 'gainsboro', font = helv10)
            self.button.grid(row=3, column = 1, padx=8, pady=8)
            self.progressbar = ttk.Progressbar(self.main, orient = HORIZONTAL, 
                                           length = 300, mode = 'determinate')
            self.progressbar.grid(row=4, column = 1, padx=5, pady=5)
            self.status = tk.Label(self.main, text="Click button to start analysis..", relief=SUNKEN, anchor=W, bd=2) 
            self.status.grid(row = 10, column = 1, padx = 20, pady=10)
           
    def open_excel(self):
        os.system("start EXCEL.EXE Final_Results.xlsx")
    
           
    def set_default(self, entryText,a, b, c, d,e,f):
        #Set default
        entryText.set( "43.71%" )
        a.set("20.06%" )
        b.set("12.64%")
        c.set( "4.52%" )
        d.set( "5.12%" )
        e.set( "3.36%" )
        f.set( "10.59%") 
    
    def start_action(self):
        self.button.config(state=tk.DISABLED)
        thread = AsyncAction(self.queue, 42, self.data_path, self.Func, self.Lead, 
                             self.Commodity, self.Life,self.Demand, self.Price, self.Resp, self.status, self.progressbar)
        thread.start()
        self.poll_thread(thread)

    def poll_thread(self, thread):
        self.check_queue()
        if thread.is_alive():
            self.after(100, lambda: self.poll_thread(thread))
        else:
            self.button.config(state=tk.NORMAL)
            helv10 = tkFont.Font(family='Helvetica', size=10, weight=tkFont.BOLD)
            self.button = tk.Button(self.main,text = 'See Analysis Results', 
                                command = self.open_excel, width = 20, bg = 'gainsboro', font = helv10)
            self.button.grid(row=7, column = 1, padx=8, pady=8)
            
    def check_queue(self):
        while self.queue.qsize():
            try:
                step = self.queue.get(0)
                self.progressbar.step(step * 100)
            except queue.Empty:
                pass

class AsyncAction(threading.Thread):
    def __init__(self, queue, steps, data_path, Func, Lead, Commodity, Life, Demand,Price, Resp, status, progressbar ):
        super().__init__()
        self.queue = queue
        self.steps = steps
        self.path = data_path
        self.func = float(Func[:-1])/100
        self.lead = float(Lead[:-1])/100
        self.commod = float(Commodity[:-1])/100
        self.Life = float(Life[:-1])/100
        self.demand = float(Demand[:-1])/100
        self.price = float(Price[:-1])/100
        self.resp = float(Resp[:-1])/100
        self.status = status
        self.progressbar = progressbar
        
    def get_bootstrap(self,row):
        test_size= 10000
        CI= 0.99
        testing = [0]*test_size
        values = row[6:].tolist()
        lead_time = row['LeadTime']
        for i in range(0,test_size):
            testing[i] = sum(random.sample(values, lead_time ))
        testing.sort()
        return testing[int(test_size*CI-1)]

    def get_method(self,row):
        if (row["DailyMean"]*1.1) > row['DailyVariance']:
            return "Poisson"
        else:
            return "Bootstrapping"
    
    def get_safety_stock(self,row):
        if (row['LeadTime']*row['DailyMean']) > 0.052:
            return int(round(row['LeadTime']*row['DailyMean'] + 1.64 * np.sqrt(row['LeadTime']*row['DailyMean'])))
        else:
            return 0
    
    def consumption_label(self, row):
        #Create consumption label column 

        if row['CreatedOn'] > self.reference_Date:
            return "Created Recently"
        else:
            if 'Year5' in row:
                if (row['Year0'] > 299 or row["Year1"] > 299 or row["Year2"] > 299 or row["Year3"] > 299 
                    or row["Year4"] > 299 or row['Year5'] >299):
                    return "Mass Consumption"
                else:
                    if (row['Year0'] > 0 or row["Year1"] > 0 or row["Year2"] > 0 or row["Year3"] > 0 
                        or row["Year4"] > 0 or row['Year5'] >0):
                        return "Low Consumption"
                    else:
                        return "Very Low Consumption"
            else:
                if (row['Year0'] > 299 or row["Year1"] > 299 or row["Year2"] > 299 or row["Year3"] > 299 
                    or row["Year4"] > 299):
                    return "Mass Consumption"
                else:
                    if (row['Year0'] > 0 or row["Year1"] > 0 or row["Year2"] > 0 or row["Year3"] > 0 
                        or row["Year4"] > 0):
                        return "Low Consumption"
                    else:
                        return "Very Low Consumption"

    #Calculate Min and Max
    def get_AHMScore(self,row):
        return ((row['Functionality']*self.func) + (row['ResponseTime']*self.resp) + (row['LeadTime']*self.lead) 
                + (row['Commodity/OEM']*self.commod) + (row['LifeCycle']*self.Life) + (row['Demand']*self.demand) 
                + (row['Price']*self.price ))    

    def run(self):
        self.status['text'] = "{}".format("Compiling data files")
        inventory_files = self.path + "\Inventory*"
        
        all_inventories = pd.read_excel(glob.glob(inventory_files)[0])

        usage_files = self.path + r"\Issues*"
        all_usage = pd.read_excel(glob.glob(usage_files)[0])

        self.progressbar['value'] +=5

        #Calculate yearly usage
        self.status['text'] = "{}".format("Calculating annual usage")
        yearly_usage = all_usage[['UniqueID', 'EntryDate', 'Qty']]
        yearly_usage['Year'] = pd.DatetimeIndex( yearly_usage['EntryDate']).year
        yearly_usage = yearly_usage.groupby(['UniqueID','Year']).sum().reset_index()


        #Add the yearly usage columns
        years = yearly_usage.Year.unique()
        number_years = len(years)
        usage = [0]*number_years
        for x, i in enumerate(years):
            usage[x] = yearly_usage[yearly_usage['Year'] == i][['UniqueID', "Qty"]]
            all_inventories["Year" + str(x)] = all_inventories['UniqueID'].map(usage[x].set_index('UniqueID')['Qty']).fillna(0)


        #Get Daily Usage
        self.status['text'] = "{}".format("Calculating daily usage")
        all_usage =all_usage[['UniqueID', 'EntryDate', 'Qty']]


        self.reference_Date = max(all_usage['EntryDate']) - timedelta(days=365*3)
        all_inventories['Consumption_Label'] = all_inventories.apply(lambda row: self.consumption_label(row), axis=1)

        self.progressbar['value'] +=10

        #Sort the dates
        all_usage.sort_values(by = 'EntryDate')
        #Pivot table format
        all_usage = pd.pivot_table(all_usage, values = 'Qty', index= 'UniqueID',columns = 'EntryDate', aggfunc=np.sum, fill_value= 0)
        #create a column with UniqueID and move to column 1
        all_usage['UniqueID'] = all_usage.index
        cols = list(all_usage.columns)
        cols = [cols[-1]] + cols[:-1]
        all_usage = all_usage[cols]

        #Include all days in the date range 
        #get number of days between the first and last date #generalize start and end date
        days_list = pd.date_range(str(min(years))+'-01-01 00:00:00', periods=(date(max(years), 12, 31) - date(min(years), 1, 1)).days).tolist()
        cols = ['UniqueID'] + days_list
        all_days= pd.DataFrame(columns = cols)
        cols = ['UniqueID'] + days_list

        all_days['UniqueID'] = all_usage['UniqueID']
        for i in cols[1:]:
            if pd.Timestamp(i) in all_usage.columns:
                all_days[i] = all_usage[i]
            else:
                all_days[i] = 0

        all_days = all_days.fillna(0) 

        self.progressbar['value'] +=15

        all_days['DailyMean'] = all_days.iloc[:, 1:].mean(axis=1)
        all_days['DailyVariance'] = all_days.iloc[:, 1:].var(axis=1)
        all_days['DailyMax'] = all_days.iloc[:, 1:].max(axis=1)
        all_days['DailyMax'] = all_days['DailyMax'].astype(int)

        all_days['Method'] = all_days.apply(lambda row: self.get_method(row), axis=1)

        #Add lead time
        lead_time = all_inventories[['UniqueID', 'LeadTime']]
        all_days = pd.merge(all_days, lead_time, how='inner', on= 'UniqueID')

        #Poisson
        self.status['text'] = "{}".format("Calculating safety stock - Poisson")
        Poisson = all_days[all_days['Method']=="Poisson"] 

        Poisson['SafetyStock'] = Poisson.apply(lambda row: self.get_safety_stock(row), axis=1) 

        self.progressbar['value'] +=5
        #Bootstrap
        self.status['text'] = "{}".format("Calculating safety stock - Bootstrap")
        Bootstrap = all_days[all_days['Method']=="Bootstrapping"] 

        cols = list(Bootstrap.columns)
        cols = [cols[0]] + cols[-5:] + cols[1:-5]
        Bootstrap = Bootstrap[cols]
        Bootstrap['SafetyStock'] = Bootstrap.apply(lambda row: self.get_bootstrap(row), axis=1) 

        self.progressbar['value'] +=50

        #Combine Bootstrap and Poisson
        self.status['text'] = "{}".format("Finalizing low consumption analysis")
        Bootstrap_Poisson = pd.concat([Poisson, Bootstrap])

        ##Low Consumption Analysis
        Low_Consumption = all_inventories[all_inventories['Consumption_Label'] == "Low Consumption"]
        Low_Consumption = Low_Consumption[['UniqueID', 'MovAvgPrice']]
        LowConsAnalysis = pd.merge(Low_Consumption, Bootstrap_Poisson, how = 'left', on = 'UniqueID')

        #Calculate Min and Max 
        LowConsAnalysis['SuggestedMin'] = round((LowConsAnalysis['LeadTime'] * LowConsAnalysis['DailyMean']) + 
                                         LowConsAnalysis['SafetyStock'], 0).astype(int)
        LowConsAnalysis['EOQ'] = round(np.sqrt((2*LowConsAnalysis['DailyMean']*365)/(0.3*LowConsAnalysis['MovAvgPrice'])), 2)
        LowConsAnalysis['2YrDemand'] = np.ceil(LowConsAnalysis['DailyMean']*2*365)
        LowConsAnalysis['ModEOQ'] = np.ceil(LowConsAnalysis[['EOQ', '2YrDemand']].min(axis=1))
        LowConsAnalysis['SuggestedMax'] = LowConsAnalysis['SuggestedMin'] + LowConsAnalysis['ModEOQ'].astype(int)

        ##Mass Consumption Analysis
        self.status['text'] = "{}".format("Finalizing mass consumption analysis")
        Mass_Consumption = all_inventories[all_inventories['Consumption_Label'] == "Mass Consumption"]
        Mass_Consumption = Mass_Consumption[['UniqueID', 'MovAvgPrice']]
        MassConsAnalysis = pd.merge(Mass_Consumption, Bootstrap_Poisson, how = 'left', on = 'UniqueID')

        #Calculate Min and Max 
        Constant = 1.65
        MassConsAnalysis['StdDev'] = np.sqrt(MassConsAnalysis['DailyVariance'])
        MassConsAnalysis['SafetyStock'] = round(np.sqrt(MassConsAnalysis['LeadTime'] + 
                                                        ((MassConsAnalysis['DailyMean']**2)*(MassConsAnalysis['StdDev']**2)))*Constant, 0)
        MassConsAnalysis['SuggestedMin'] = round((MassConsAnalysis['LeadTime']*MassConsAnalysis['DailyMean']) + MassConsAnalysis['SafetyStock'],0).astype(int)
        MassConsAnalysis['EOQ'] = round(np.sqrt((2*MassConsAnalysis['DailyMean']*365)/(0.3*MassConsAnalysis['MovAvgPrice'])), 0)
        MassConsAnalysis['SuggestedMax'] = MassConsAnalysis['SuggestedMin'] + MassConsAnalysis['EOQ'].astype(int)

        self.progressbar['value'] +=5

        ##Very Low Consumption
        self.status['text'] = "{}".format("Finalizing very low consumption analysis")
        VeryLowConsAnalysis = all_inventories[all_inventories['Consumption_Label'] == "Very Low Consumption"]
        VeryLowConsAnalysis['ABC_Classification'] = VeryLowConsAnalysis.apply(lambda row: 'A' if row['MovAvgPrice'] > 1000 else ('B' if row['MovAvgPrice']>25 else 'C'), axis=1)
        lower = VeryLowConsAnalysis.columns.tolist().index('Year0')
        if 'Year5' in VeryLowConsAnalysis:
            upper = VeryLowConsAnalysis.columns.tolist().index('Year5') + 1
        else:
            upper =VeryLowConsAnalysis.columns.index('Year4') + 1
        VeryLowConsAnalysis['XYZ_Classification'] = VeryLowConsAnalysis.apply(lambda row: 'X' if np.var(row[lower:upper]) <= 1 else ('Y' if np.var(row[lower:upper]) <= 5 else 'Z'), axis=1)
        VeryLowConsAnalysis['Functionality'] = 4
        VeryLowConsAnalysis['ResponseTime'] = 2
        VeryLowConsAnalysis['LeadTime'] = 5
        VeryLowConsAnalysis['Commodity/OEM'] = 5
        VeryLowConsAnalysis['LifeCycle'] = 1
        VeryLowConsAnalysis['Demand'] = VeryLowConsAnalysis.apply(lambda row: 5 if row['XYZ_Classification'] == 'Z' else (2 if row['XYZ_Classification'] == 'Y' else 1), axis = 1)
        VeryLowConsAnalysis['Price'] = VeryLowConsAnalysis.apply(lambda row: 1 if row['ABC_Classification'] == 'A' else (2 if row['ABC_Classification'] == 'B' else 5), axis = 1)

        VeryLowConsAnalysis['AHM_Score'] = VeryLowConsAnalysis.apply(lambda row: self.get_AHMScore(row), axis=1)
        VeryLowConsAnalysis['VED_Label'] = VeryLowConsAnalysis.apply(lambda row: "Vital" if row['AHM_Score'] >= 4 else ("Essential" if row['AHM_Score'] >= 3 else "Desirable"), axis=1)
        VeryLowConsAnalysis['SuggestedMin'] = VeryLowConsAnalysis.apply(lambda row: 1 if row['VED_Label'] == "Vital" else 0, axis =1)
        VeryLowConsAnalysis['SuggestedMax'] = VeryLowConsAnalysis.apply(lambda row: 2 if row['SuggestedMin'] == 1 else 0, axis=1)

        MinMaxVals = pd.concat([LowConsAnalysis[['UniqueID', 'SuggestedMin', 'SuggestedMax']], VeryLowConsAnalysis[['UniqueID', 'SuggestedMin', 'SuggestedMax']], MassConsAnalysis[['UniqueID', 'SuggestedMin', 'SuggestedMax']]])

        self.status['text'] = "{}".format("Compiling analysis")
        Final_results = pd.merge(all_inventories, MinMaxVals, how = 'left', on = 'UniqueID')


        self.status['text'] = "{}".format("Downloading analysis")
        if Final_results.shape[0] < 50000:
            Final_results.to_excel("Final_Results.xlsx", engine='xlsxwriter')
        else:    
            Final_results.iloc[:50000, :].to_excel("Final_Results.xlsx", engine='xlsxwriter')
            Final_results.iloc[50000:100000, :].to_excel("Final_Results2.xlsx", engine='xlsxwriter')
            Final_results.iloc[100000:, :].to_excel("Final_Results3.xlsx", engine='xlsxwriter')
        self.status['text'] = "{}".format("Analysis Results Downloaded")
        self.progressbar['value'] +=10

if __name__ == "__main__":
    app = App()
    app.mainloop()


C:\Users\salma.katri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\salma.katri\AppData\Local\Continuum\anaconda3\lib\threading.py:917: FutureWarning: 'UniqueID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  self.run()
C:\Users\salma.katri\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\salma.kat